# Init

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import itertools
import inspect
from pprint import pprint

import numpy as np

from labcore.measurement import *

# Musings

The way we would like to measure looks something like this:

```python
    >>> m = Measurement(
    ...     setup_function,
    ...     sweep_object,
    ...     teardown_function
    ... )
    ... m.run()
```

The reason why we would like to define the measurement in an object is to be able to have access to main characteristics (such as the type of data acquired during the run) in advance, before any measurement code is executed. This will also be useful for serializing measurements.

The most tricky aspect here is to define what the ``sweep`` object is. Often we define the flow in an experiment by directly using for-loops and the like. But that will not allow us the desired goals. We need to come up with an object-based description that makes the essence of the flow clear without executing it.

## Basic rules for individual sweep behavior:

- a sweep object can be iterated over. each step performs some actions, and returns some data.
  that means, we want to be able to use it like this:
  
    ```python
      >>> for data in sweep_object:
      ...    print(data)
      {variable_1: some_data, variable_2: more_data}
      {variable_1: different_data, variable_2: more_different_data}
    ```
 
 
- a sweep object is defined by a pointer and 1 or more actions. 
    - A pointer is again an iterable, and the coordinates it traverses may or may not be known in advance. 
    - Actions are callables that take the values the pointer is returning at each iteration as arguments. Each action is executed once for each pointer.
  
  A simple sweep can then be defined like this:
    
    ```python
        >>> sweep_object = Sweep(range(5), func_1, func_2)
        
    ```
  
  Executing this sweep will loop over the range, and will then call ``func_1(i)`` and subsequently ``func_2(i)``, for each ``i`` in the range from 0 to 4.
  

## Specifying return data

When iterating over a sweep, annotated data is returned. For that to work, we add the corresponding meta information to each element that produces data. Typically, actions are functions that we use to set parameters and acquire signals, and thus produce data that we want to record. But also pointers can produce data, as we will see later.

We can annotate a function that produces data using the ``record_output`` decorator:

```python
   >>> @record_output(
   ...     DataSpec(name='x', unit='V'),
   ...     DataSpec(name='y', unit='A', depends_on=['x']),
   ... )
   ... def measure_current(voltage):
   ...     return voltage, 50./voltage)
   ... measure_current(1)
   {'x': 1, 'y': 50}
```

``DataSpec`` contains more information than just the name -- this is not immediately relevant, but important meta data we will need for faithfully storing the data.

More generally we can use the ``record`` function to annotate outputs. See examples below.

## dependents and independents

TBD

# Examples

## Annotating objects for recording

In [3]:
# defining some example measurement functions without short-hand notations

@record_func_output(DataSpec('x'), DataSpec('y', depends_on=['x'], type='array'))
def measure_stuff(n, *args, **kwargs):
    return n, np.random.normal(size=n)

@record_func_output(DataSpec('a'))
def set_stuff(x, *args, **kwargs):
    return x

measure_stuff(1), set_stuff(1)

({'x': 1, 'y': array([-0.59147468])}, {'a': 1})

In [4]:
# we can also annotate generators

@record_func_output(ds('a'))
def make_sequence(n):
    for i in range(n):
        yield i
        
for data in make_sequence(3):
    print(data)

{'a': 0}
{'a': 1}
{'a': 2}


In [5]:
# using ``record`` is a practical way of annotating records
# just before executing, independently of an earlier function definition

def get_some_data(n):
    return np.random.normal(size=n)

record(ds('rand'), get_some_data)(3)

{'rand': array([1.43446288, 0.73257132, 1.38934264])}

In [6]:
# record also provides a simple short hand for labelling regular iterables and iterators:

for data in record(ds('x'), zip(np.linspace(0,1,6), np.arange(6))):
    print(data)
    
for data in record(ds('x'), ds('y'), zip(np.linspace(0,1,6), np.arange(6))):
    print(data)
    
for data in record(ds('x'), ds('y'), np.linspace(0,1,6)):
    print(data)

{'x': 0.0}
{'x': 0.2}
{'x': 0.4}
{'x': 0.6000000000000001}
{'x': 0.8}
{'x': 1.0}
{'x': 0.0, 'y': 0}
{'x': 0.2, 'y': 1}
{'x': 0.4, 'y': 2}
{'x': 0.6000000000000001, 'y': 3}
{'x': 0.8, 'y': 4}
{'x': 1.0, 'y': 5}
{'x': 0.0, 'y': None}
{'x': 0.2, 'y': None}
{'x': 0.4, 'y': None}
{'x': 0.6000000000000001, 'y': None}
{'x': 0.8, 'y': None}
{'x': 1.0, 'y': None}


## Single sweeps

### Effect of annotating output

In [7]:
# important note: without record annotations sweeps still are executed, but no data is recorded.

sweep_object = Sweep(range(3), lambda x: np.random.normal(size=x))
for data in sweep_object:
    print(data)

{}
{}
{}


In [8]:
sweep_object = Sweep(range(3), record(ds('some_array'), lambda x: np.random.normal(size=x)))

for data in sweep_object:
    print(data)

{'some_array': array([], dtype=float64)}
{'some_array': array([0.99710995])}
{'some_array': array([-0.32991877,  0.81074413])}


In [9]:
sweep_object = Sweep(record(ds('x'), range(3)), 
                     record(ds('rand_arr'), lambda x: np.random.normal(size=x)))

for data in sweep_object:
    print(data)

{'x': 0, 'rand_arr': array([], dtype=float64)}
{'x': 1, 'rand_arr': array([-1.28838666])}
{'x': 2, 'rand_arr': array([-1.53734116, -0.74467527])}


In [10]:
# if the pointer is annotated, we need to match arguments that are supposed to be passed.
# otherwise a default value is passed.

sweep_object = Sweep(record(ds('y'), range(3)), 
                     record(ds('rand_arr'), lambda x: x))

for data in sweep_object:
    print(data)

{'y': 0, 'rand_arr': None}
{'y': 1, 'rand_arr': None}
{'y': 2, 'rand_arr': None}


In [11]:
# The examples above are a bit artificial to highlight basic principles.
# A more typical example as we might see it in the lab, that looks much more elegant already.
# Notes:
# - the sweep parameter is added as an independent to the measurement return
#   this is done dynamically, so we can reuse actions easily with different pointers
#   it only affects parameters that are specified as dependent (i.e., where ``depends_on`` is not ``None``)

@record_func_output(ds('y', unit='A', depends_on=[]))
def my_measurement():
    print('acquiring current...')
    return np.random.normal()

In [12]:
sweep = sweep_parameter(ds('x', unit='V'), np.linspace(-0.2, 0.2, 6), my_measurement)
pprint(sweep.get_data_specs())

for data in sweep:
    print(data)

[DataSpec(name='x', depends_on=None, type=<DataType.scalar: 'scalar'>, unit='V'),
 DataSpec(name='y', depends_on=['x'], type=<DataType.scalar: 'scalar'>, unit='A')]
acquiring current...
{'x': -0.2, 'y': 0.8451472946459849}
acquiring current...
{'x': -0.12000000000000001, 'y': -1.2663124066041591}
acquiring current...
{'x': -0.04000000000000001, 'y': 0.310708842858522}
acquiring current...
{'x': 0.03999999999999998, 'y': -0.10926332758745423}
acquiring current...
{'x': 0.12, 'y': 0.12893070611534133}
acquiring current...
{'x': 0.2, 'y': 0.40911946386164755}


In [13]:
sweep = sweep_parameter(ds('not_x', unit='V'), np.linspace(-0.2, 0.2, 6), my_measurement)
pprint(sweep.get_data_specs())

for data in sweep:
    print(data)

[DataSpec(name='not_x', depends_on=None, type=<DataType.scalar: 'scalar'>, unit='V'),
 DataSpec(name='y', depends_on=['not_x'], type=<DataType.scalar: 'scalar'>, unit='A')]
acquiring current...
{'not_x': -0.2, 'y': -0.11855346976213665}
acquiring current...
{'not_x': -0.12000000000000001, 'y': -0.3495706002298027}
acquiring current...
{'not_x': -0.04000000000000001, 'y': 0.03195130451904915}
acquiring current...
{'not_x': 0.03999999999999998, 'y': 0.051803810438715}
acquiring current...
{'not_x': 0.12, 'y': -0.08047228243725238}
acquiring current...
{'not_x': 0.2, 'y': 2.154884250929689}


# TESTING